In [ ]:
%%writefile requirements.txt
absl-py==2.1.0
aiohappyeyeballs==2.4.3
aiohttp==3.10.10
aiosignal==1.3.1
alpha_vantage==3.0.0
astunparse==1.6.3
attrs==24.2.0
beautifulsoup4==4.12.3
blinker==1.8.2
certifi==2024.8.30
charset-normalizer==3.4.0
click==8.1.7
Flask==3.0.3
flatbuffers==24.3.25
frozendict==2.4.6
frozenlist==1.4.1
gast==0.6.0
google-pasta==0.2.0
grpcio==1.67.0
h5py==3.12.1
html5lib==1.1
idna==3.10
itsdangerous==2.2.0
Jinja2==3.1.4
joblib==1.4.2
keras==3.6.0
libclang==18.1.1
lxml==5.3.0
Markdown==3.7
markdown-it-py==3.0.0
MarkupSafe==3.0.2
mdurl==0.1.2
ml-dtypes==0.4.1
multidict==6.1.0
multitasking==0.0.11
namex==0.0.8
numpy==1.26.4
opt_einsum==3.4.0
optree==0.13.0
packaging==24.1
panadas==0.2
pandas==2.2.3
pandas_ta==0.3.14b0
peewee==3.17.7
platformdirs==4.3.6
propcache==0.2.0
protobuf==4.25.5
Pygments==2.18.0
python-dateutil==2.9.0.post0
python-dotenv==1.0.1
pytz==2024.2
requests==2.32.3
rich==13.9.2
scikit-learn==1.5.2
scipy==1.13.1
setuptools==75.2.0
six==1.16.0
soupsieve==2.6
tensorboard==2.17.1
tensorboard-data-server==0.7.2
tensorflow==2.17.0
termcolor==2.5.0
threadpoolctl==3.5.0
typing_extensions==4.12.2
tzdata==2024.2
urllib3==2.2.3
webencodings==0.5.1
Werkzeug==3.0.4
wheel==0.44.0
wrapt==1.16.0
yarl==1.15.5

# Step 1 :- Making a requirement text file

In [ ]:
! pip install -r requirements.txt

# Installing Requirements

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import pandas_ta as ta
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.optimizers import Adam

def preprocess_data(df):
    # Ensure all data is numeric and handle non-numeric values
    df['close'] = pd.to_numeric(df['close'], errors='coerce')
    df = df.dropna(subset=['close'])

    # Add technical indicators
    df['SMA'] = ta.sma(df['close'], length=20)  # Simple Moving Average
    df['EMA'] = ta.ema(df['close'], length=20)  # Exponential Moving Average
    df['RSI'] = ta.rsi(df['close'], length=14)  # Relative Strength Index

    # Drop rows with NaN values (due to indicator calculations)
    df = df.dropna()

    # Normalize the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(df[['close', 'SMA', 'EMA', 'RSI']])
    return scaled_data, scaler

def create_sequences(data, sequence_length=60):
    # preparing sequences for LSTM model
    sequences = []
    targets = []
    for i in range(sequence_length, len(data)):
        sequences.append(data[i-sequence_length:i, :])
        targets.append(data[i, 0])
    X = np.array(sequences)
    y = np.array(targets)
    X = np.reshape(X, (X.shape[0], X.shape[1], X.shape[2]))
    return X, y

def build_bidirectional_lstm_model(input_shape, units=50, dropout_rate=0.2, learning_rate=0.001):
    model = Sequential()
    model.add(Bidirectional(LSTM(units=units, return_sequences=True), input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(units=units, return_sequences=True)))
    model.add(Dropout(dropout_rate))
    model.add(Bidirectional(LSTM(units=units, return_sequences=False)))
    model.add(Dropout(dropout_rate))
    model.add(Dense(units=1))
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    return model

def train_lstm_model(X, y, input_shape, epochs=50, batch_size=32):
    model = build_bidirectional_lstm_model(input_shape)
    model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2)
    model.save('lstm_model.h5')
    return model


# Generated LSTM Model 

In [ ]:
import sqlite3

def connect_db():
    return sqlite3.connect('predictions.db')

def create_table():
    with connect_db() as conn:
        conn.execute('''CREATE TABLE IF NOT EXISTS predictions
                        (id INTEGER PRIMARY KEY AUTOINCREMENT,
                         symbol TEXT NOT NULL,
                         predicted_price REAL NOT NULL,
                         date TEXT NOT NULL,
                         timestamp DATETIME DEFAULT CURRENT_TIMESTAMP)''')

def store_prediction(symbol, predicted_price, date):
    with connect_db() as conn:
        conn.execute('INSERT INTO predictions (symbol, predicted_price, date) VALUES (?, ?, ?)',
                     (symbol, predicted_price, date))
        conn.commit()

def query_db(query, args=(), one=False):
    with connect_db() as conn:
        conn.row_factory = sqlite3.Row
        cur = conn.execute(query, args)
        rv = cur.fetchall()
        return (rv[0] if rv else None) if one else rv


# Created sqlite database for tracking 

In [ ]:
import requests
import pandas as pd

def fetch_live_data(symbol, api_key):
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={api_key}&outputsize=full'
    response = requests.get(url)
    data = response.json()

    # Check if the response contains an error message
    if "Error Message" in data:
        raise ValueError(f"Error fetching data for symbol {symbol}: {data['Error Message']}")
    if "Information" in data:
        raise ValueError(f"Information: {data['Information']}")

    # Extract time series data
    if "Time Series (Daily)" not in data:
        raise ValueError(f"Unexpected data format for symbol {symbol}")

    time_series = data["Time Series (Daily)"]
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df = df.rename(columns={
        "1. open": "open",
        "2. high": "high",
        "3. low": "low",
        "4. close": "close",
        "5. volume": "volume"
    })
    df.index.name = 'Date'
    df = df.reset_index()
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.sort_values('Date')

    return df


# Using Api to get stock prices

In [ ]:
import os

# Set your environment variables
os.environ['ALPHA_VANTAGE_API_KEY'] = '$(YOUR_ALPHA_VANTAGE_API_KEY)'
os.environ['NGROK_AUTHTOKEN'] = '$(NGROK_AUTHTOKEN)'


# Importing environment variable

In [ ]:
pip install --upgrade pyngrok

# Double check if pyngrok version is latest or not

In [ ]:
import os
import numpy as np
from flask import Flask, render_template, request
from lstm_model import preprocess_data, create_sequences, train_lstm_model
from alpha_vantage_data import fetch_live_data
from tensorflow.keras.models import load_model
from database import store_prediction, query_db
from datetime import datetime
from dotenv import load_dotenv
from pyngrok import ngrok

# Load environment variables from .env file
load_dotenv()

# Set ngrok authtoken
NGROK_AUTHTOKEN = os.getenv('NGROK_AUTHTOKEN')
if not NGROK_AUTHTOKEN:
    raise ValueError("Please set the NGROK_AUTHTOKEN environment variable in the .env file.")
ngrok.set_auth_token(NGROK_AUTHTOKEN)

app = Flask(__name__)

# Optionally, remove flask_ngrok if not needed
# from flask_ngrok import run_with_ngrok
# run_with_ngrok(app)

# Create a single ngrok tunnel
public_url = ngrok.connect(5000)
print("ngrok tunnel URL:", public_url)

# Load your Alpha Vantage API key from environment variable
API_KEY = os.getenv('ALPHA_VANTAGE_API_KEY')

if not API_KEY:
    raise ValueError("Please set the ALPHA_VANTAGE_API_KEY environment variable in the .env file.")

@app.route('/', methods=['GET', 'POST'])
def index():
    prediction = None
    error = None
    symbol = None

    if request.method == 'POST':
        symbol = request.form.get('symbol', '').upper().strip()

        if not symbol:
            error = "Please enter a valid stock symbol."
            return render_template('index.html', prediction=None, error=error)

        try:
            # Fetch live stock data using Alpha Vantage
            df = fetch_live_data(symbol, API_KEY)
            if df.empty:
                raise ValueError(f"No data found for symbol: {symbol}")

            # Preprocess data
            scaled_data, scaler = preprocess_data(df)

            # Create sequences
            sequence_length = 60
            X, y = create_sequences(scaled_data, sequence_length=sequence_length)

            # Check if model exists; if not, train a new one
            model_path = 'lstm_model.h5'
            if os.path.exists(model_path):
                model = load_model(model_path)
            else:
                input_shape = (X.shape[1], X.shape[2])
                model = train_lstm_model(X, y, input_shape)

            # Make prediction
            last_sequence = scaled_data[-sequence_length:]
            last_sequence = np.reshape(last_sequence, (1, sequence_length, last_sequence.shape[1]))  # Adjust shape to match indicators
            predicted_price = model.predict(last_sequence)

            # Create a placeholder for the full feature set to inverse transform
            full_feature_set = np.zeros((predicted_price.shape[0], scaled_data.shape[1]))
            full_feature_set[:, 0] = predicted_price[:, 0]
            predicted_price = scaler.inverse_transform(full_feature_set)[:, 0]

            prediction = round(float(predicted_price[0]), 2)

            # Store the prediction in the database
            current_date = datetime.now().strftime('%Y-%m-%d')
            store_prediction(symbol, prediction, current_date)

        except Exception as e:
            error = str(e)
            return render_template('index.html', prediction=None, error=error)

    return render_template('index.html', prediction=prediction, error=error, symbol=symbol)

@app.route('/history')
def history():
    predictions = query_db('SELECT symbol, predicted_price, timestamp FROM predictions ORDER BY timestamp DESC')
    return render_template('history.html', predictions=predictions)


if __name__ == "__main__":
    app.run(host="0.0.0.0", port=5000)

# made a flask app which gives public address to local host running lstm model